### ДЗ #7
CNN в обработке текстов

In [1]:
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from sklearn.model_selection import train_test_split
from nltk.probability import FreqDist

[nltk_data] Downloading package punkt to /home/postas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dataset = pd.read_excel('reviews.xls')

In [3]:
dataset.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [4]:
dataset.Rating.value_counts(normalize=True)

5    0.706036
1    0.110170
4    0.103490
3    0.044097
2    0.036207
Name: Rating, dtype: float64

In [5]:
# Какая сложная разметка... 5 типов оценок

In [6]:
# Бьём на трейн-тест
X_train, X_test, y_train, y_test = train_test_split(dataset.Content, dataset.Rating, 
                                                    test_size=0.2, random_state=42, shuffle=True)

In [7]:
# Предобработаем текст

In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [9]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [10]:
X_train_prep = X_train.apply(preprocess_text)

In [11]:
X_test_prep = X_test.apply(preprocess_text)

In [12]:
X_train_prep

2680                                  удобно
844        приходить смс код вход приложение
18714                         удобно понятно
13842    удобный приложение работать отлично
11974                     удобный приложение
                        ...                 
11284                              нравиться
11964         смешно программа пугаться рута
5390            мочь скачать ошибка номер 24
860                                 сбербанк
15795                          целое отлично
Name: Content, Length: 16527, dtype: object

In [13]:
# На вебинаре разбирался count based метод кодирования слов, которые делался вручную, вычислением частотности слова
# Не очень понятно, почему было не применить TextVectorization layer из Keras.. Видимо для лучшего понимания процесса

In [14]:
train_corpus = " ".join(X_train_prep)

In [15]:
tokens = word_tokenize(train_corpus)

In [16]:
tokens[:7]

['удобно', 'приходить', 'смс', 'код', 'вход', 'приложение', 'удобно']

In [17]:
len(tokens)

82908

In [18]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [19]:
len(tokens_filtered)

82354

In [20]:
dist = FreqDist(tokens_filtered)

In [21]:
dist.most_common(5)

[('приложение', 4936),
 ('удобно', 2606),
 ('работать', 1550),
 ('удобный', 1413),
 ('отлично', 1034)]

In [22]:
max_words = 20000
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [23]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'телефон',
 'супер']

In [24]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, start=1)).items()}

In [25]:
vocabulary

{'приложение': 1,
 'удобно': 2,
 'работать': 3,
 'удобный': 4,
 'отлично': 5,
 'нравиться': 6,
 'отличный': 7,
 'хороший': 8,
 'телефон': 9,
 'супер': 10,
 'обновление': 11,
 'быстро': 12,
 'пароль': 13,
 'мочь': 14,
 'антивирус': 15,
 'банк': 16,
 'пользоваться': 17,
 'вход': 18,
 'сбербанк': 19,
 'устраивать': 20,
 'раз': 21,
 'карта': 22,
 'прошивка': 23,
 'рута': 24,
 'проблема': 25,
 'сделать': 26,
 'перевод': 27,
 'разработчик': 28,
 'программа': 29,
 'приходиться': 30,
 'ошибка': 31,
 'писать': 32,
 'счёт': 33,
 'вводить': 34,
 'норма': 35,
 'деньга': 36,
 'нормально': 37,
 'платёж': 38,
 'постоянно': 39,
 'довольный': 40,
 'около': 41,
 'код': 42,
 'исправить': 43,
 'функция': 44,
 'смс': 45,
 'понятно': 46,
 'последний': 47,
 'свой': 48,
 'вылетать': 49,
 'стать': 50,
 'шаблон': 51,
 'зайти': 52,
 'делать': 53,
 'мобильный': 54,
 'возможность': 55,
 'иня': 56,
 'приходить': 57,
 'класс': 58,
 'право': 59,
 'проверка': 60,
 'заходить': 61,
 'установить': 62,
 'meizu': 63,
 'roo

In [26]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [27]:
max_len = 100
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train_prep], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test_prep], dtype=np.int32)

In [28]:
x_train

array([[  0,   0,   0, ...,   0,   0,   2],
       [  0,   0,   0, ...,  42,  18,   1],
       [  0,   0,   0, ...,   0,   2,  46],
       ...,
       [  0,   0,   0, ...,  31,  80, 223],
       [  0,   0,   0, ...,   0,   0,  19],
       [  0,   0,   0, ...,   0, 111,   5]], dtype=int32)

In [29]:
x_train.shape

(16527, 100)

In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping  
from tensorflow.keras.utils import to_categorical

In [53]:
num_classes = 5
to_categorical(y_train, num_classes=num_classes)

IndexError: index 5 is out of bounds for axis 1 with size 5

In [41]:
to_categorical(y_train)

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [43]:
y_train.value_counts(normalize=False)

5    11654
1     1822
4     1735
3      724
2      592
Name: Rating, dtype: int64

In [44]:
# Странно, почему one_hot делает 6 классов (типа нулевой класс подразумевает?)

In [45]:
yy_train = y_train-1

In [46]:
yy_train.value_counts()

4    11654
0     1822
3     1735
2      724
1      592
Name: Rating, dtype: int64

In [47]:
to_categorical(yy_train)

array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [48]:
# Нда.. похоже на то

In [50]:
y_train_ohe = to_categorical(y_train-1, num_classes=num_classes)
y_train_ohe

array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [51]:
y_test_ohe = to_categorical(y_test-1, num_classes=num_classes)
y_test_ohe

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [52]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [54]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [55]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

In [57]:
history = model.fit(x_train, y_train_ohe,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
233/233 [==============================] - 22s 72ms/step - loss: 1.0604 - accuracy: 0.6904 - val_loss: 0.6668 - val_accuracy: 0.7641
Epoch 2/10
233/233 [==============================] - 13s 56ms/step - loss: 0.5991 - accuracy: 0.7932 - val_loss: 0.6462 - val_accuracy: 0.7731
Epoch 3/10
233/233 [==============================] - 13s 57ms/step - loss: 0.5085 - accuracy: 0.8256 - val_loss: 0.6593 - val_accuracy: 0.7683


In [58]:
score = model.evaluate(x_test, y_test_ohe, batch_size=64, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

65/65 [==============================] - 1s 22ms/step - loss: 0.6592 - accuracy: 0.7776


Test score: 0.6592420339584351
Test accuracy: 0.7775895595550537


Это видимо бэйзлайн

### Word2Vec Эмбеддинги 

In [59]:
from gensim.models import Word2Vec

/home/postas/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [76]:
# Учим word2vec
w2v = Word2Vec(sentences=X_train_prep.apply(str.split), vector_size=128, window=5, min_count=5, workers=16)

In [77]:
w2v.wv['приложение']

array([-9.25486013e-02, -3.28496337e-01,  8.15744698e-02,  3.50679606e-01,
        2.11911723e-01, -6.82867691e-02,  2.10698813e-01,  4.67283800e-02,
       -1.15957841e-01,  3.26427370e-01,  2.98801959e-01,  3.50834131e-02,
        1.26082068e-02, -1.55851990e-01,  3.21012229e-01,  4.61890608e-01,
       -9.13201794e-02,  1.11518905e-01, -4.47989255e-01,  7.72698373e-02,
        2.68623114e-01,  4.63216305e-01, -3.14577669e-01, -4.59559292e-01,
       -3.39312792e-01,  4.98683117e-02, -3.67336959e-01,  2.28761077e-01,
        8.43928382e-02, -3.54485273e-01, -1.88199684e-01, -2.57848743e-02,
        2.87096351e-01,  1.61259733e-02, -1.62464336e-01,  4.88907248e-01,
        4.42307055e-01, -3.56661469e-01,  2.41233800e-02,  1.91430882e-01,
       -1.68309376e-01,  1.43999308e-01, -5.92234731e-02, -2.35812992e-01,
        3.16355079e-01,  2.05080304e-02, -4.74247903e-01, -6.11936375e-02,
       -4.52812109e-03,  2.61858284e-01,  2.56565839e-01,  2.29721516e-01,
        1.02898136e-01,  

In [98]:
# Чтобы проинициализировать Tensorflow эмбеддинги нам нужен numpy массив где по номерам слов (теперь понятно зачем мы таким образом процессили текст)
# Будут стоять эмбеддинги из W2V

In [122]:
pretrained = np.array([w2v.wv[key] if key in w2v.wv else np.zeros(128) for key in vocabulary.keys()])

In [123]:
pretrained.shape

(11168, 128)

In [124]:
model = Sequential()
model.add(Embedding(input_dim=11168, output_dim=128,
                    weights=[pretrained],
                    input_length=max_len,
                    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [125]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [126]:
tensorboard=TensorBoard(log_dir='./logs2', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

In [127]:
history = model.fit(x_train, y_train_ohe,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
233/233 [==============================] - ETA: 0s - loss: 1.0580 - accuracy: 0.6782

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument:  indices[49,98] = 11168 is not in [0, 11168)
	 [[node sequential_5/embedding_5/embedding_lookup (defined at <ipython-input-127-be464775d94a>:1) ]]
  (1) Invalid argument:  indices[49,98] = 11168 is not in [0, 11168)
	 [[node sequential_5/embedding_5/embedding_lookup (defined at <ipython-input-127-be464775d94a>:1) ]]
	 [[sequential_5/embedding_5/embedding_lookup/_12]]
0 successful operations.
0 derived errors ignored. [Op:__inference_test_function_6260]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_5/embedding_5/embedding_lookup:
 sequential_5/embedding_5/embedding_lookup/6139 (defined at /home/postas/anaconda3/lib/python3.8/contextlib.py:113)

Input Source operations connected to node sequential_5/embedding_5/embedding_lookup:
 sequential_5/embedding_5/embedding_lookup/6139 (defined at /home/postas/anaconda3/lib/python3.8/contextlib.py:113)

Function call sta

In [128]:
# Так... учиться может валидироваться - нет. видимо первое слово в vocabulary у нас имеет индекс 1... а не 0
# Попробуем сдвинуть наши претренированные эмбеддинги на 1 (добавив эмбеддинг для слова №0 в начало)

In [129]:
dummy = np.zeros(128)

In [134]:
pretrained_shifted = np.concatenate([np.array([dummy]), pretrained])

In [131]:
pretrained

array([[-0.0925486 , -0.32849634,  0.08157447, ..., -0.09233958,
        -0.21738331,  0.20137174],
       [-0.0985495 , -0.24853018,  0.09380814, ..., -0.11755178,
        -0.16697384,  0.16497894],
       [-0.08814662, -0.40196651,  0.0668055 , ..., -0.12028905,
        -0.14268678,  0.18195447],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [135]:
pretrained_shifted

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.0925486 , -0.32849634,  0.08157447, ..., -0.09233958,
        -0.21738331,  0.20137174],
       [-0.0985495 , -0.24853018,  0.09380814, ..., -0.11755178,
        -0.16697384,  0.16497894],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [136]:
pretrained_shifted.shape

(11169, 128)

In [ ]:
# Сдвинули. Попробуем снова

In [138]:
model = Sequential()
model.add(Embedding(input_dim=11169, output_dim=128,
                    weights=[pretrained_shifted],
                    input_length=max_len,
                    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [139]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [140]:
tensorboard=TensorBoard(log_dir='./logs3', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss') 

In [141]:
history = model.fit(x_train, y_train_ohe,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
233/233 [==============================] - 10s 43ms/step - loss: 1.1761 - accuracy: 0.6698 - val_loss: 0.8601 - val_accuracy: 0.7320
Epoch 2/10
233/233 [==============================] - 10s 42ms/step - loss: 0.8051 - accuracy: 0.7379 - val_loss: 0.7638 - val_accuracy: 0.7381
Epoch 3/10
233/233 [==============================] - 10s 42ms/step - loss: 0.7520 - accuracy: 0.7433 - val_loss: 0.7380 - val_accuracy: 0.7429
Epoch 4/10
233/233 [==============================] - 10s 43ms/step - loss: 0.7069 - accuracy: 0.7619 - val_loss: 0.7328 - val_accuracy: 0.7447
Epoch 5/10
233/233 [==============================] - 10s 43ms/step - loss: 0.7091 - accuracy: 0.7556 - val_loss: 0.7314 - val_accuracy: 0.7423
Epoch 6/10
233/233 [==============================] - 10s 43ms/step - loss: 0.7128 - accuracy: 0.7565 - val_loss: 0.7323 - val_accuracy: 0.7471


In [143]:
# Попробуем доучить еще без eraly stop-а
model.fit(x_train, y_train_ohe,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/10
233/233 [==============================] - 10s 42ms/step - loss: 0.7077 - accuracy: 0.7565 - val_loss: 0.7391 - val_accuracy: 0.7399
Epoch 2/10
233/233 [==============================] - 10s 42ms/step - loss: 0.7049 - accuracy: 0.7575 - val_loss: 0.7304 - val_accuracy: 0.7453
Epoch 3/10
233/233 [==============================] - 10s 42ms/step - loss: 0.7038 - accuracy: 0.7583 - val_loss: 0.7222 - val_accuracy: 0.7526
Epoch 4/10
233/233 [==============================] - 10s 42ms/step - loss: 0.7013 - accuracy: 0.7592 - val_loss: 0.7232 - val_accuracy: 0.7538
Epoch 5/10
233/233 [==============================] - 10s 43ms/step - loss: 0.7010 - accuracy: 0.7612 - val_loss: 0.7272 - val_accuracy: 0.7453
Epoch 6/10
233/233 [==============================] - 10s 43ms/step - loss: 0.6997 - accuracy: 0.7594 - val_loss: 0.7277 - val_accuracy: 0.7441
Epoch 7/10
233/233 [==============================] - 10s 43ms/step - loss: 0.6951 - accuracy: 0.7613 - val_loss: 0.7366 - val_accuracy:

In [144]:
score = model.evaluate(x_test, y_test_ohe, batch_size=64, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

65/65 [==============================] - 0s 4ms/step - loss: 0.7082 - accuracy: 0.7558


Test score: 0.7082298994064331
Test accuracy: 0.7558083534240723


In [145]:
# В целом хуже скор получается.. Есть мысль, что в этом случае получается много "холодных" слов.. 
# Тех, на которых W2V не учился. И для которых ембеддинги - просто нули.
# Возможно стоит разрешить эмбеддингам тоже учиться...

In [152]:
model = Sequential()
model.add(Embedding(input_dim=11169, output_dim=128,
                    weights=[pretrained_shifted],
                    input_length=max_len,
                    trainable=True))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [153]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [154]:
history = model.fit(x_train, y_train_ohe,
                    batch_size=64,
                    epochs=15,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/15
233/233 [==============================] - 13s 53ms/step - loss: 0.9280 - accuracy: 0.7164 - val_loss: 0.7147 - val_accuracy: 0.7447
Epoch 2/15
233/233 [==============================] - 12s 53ms/step - loss: 0.6390 - accuracy: 0.7734 - val_loss: 0.6705 - val_accuracy: 0.7586
Epoch 3/15
233/233 [==============================] - 12s 53ms/step - loss: 0.5225 - accuracy: 0.8096 - val_loss: 0.7088 - val_accuracy: 0.7598


In [155]:
score = model.evaluate(x_test, y_test_ohe, batch_size=64, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

65/65 [==============================] - 0s 5ms/step - loss: 0.6920 - accuracy: 0.7619


Test score: 0.6920420527458191
Test accuracy: 0.7618586421012878


In [156]:
# Ну, даже если и чуть получше, все равно хуже, чем с родными эмбеддингами от Tensorflow